In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## INTRO TO TEXT MINING PART 3 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [ ]:
#=================================================-
#### Slide 7: Loading packages  ####

# Helper packages.
from pathlib import Path
import os
import pandas as pd
import numpy as np
import pickle
# Packages with tools for text processing.
import nltk

# Packages for working with text data.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
# Packages for getting data ready for and building a LDA model.
import gensim
from gensim import corpora, models
from pprint import pprint
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim




In [ ]:
#=================================================-
#### Slide 8: Directory settings  ####

# Set 'main_dir' to location of the project folder
home_dir = Path(".").resolve()
main_dir = home_dir.parent
print(main_dir)
data_dir = str(main_dir) + "/data"
plot_dir = str(main_dir) + "/plots"
print(data_dir)
os.chdir(data_dir)




In [ ]:
#=================================================-
#### Slide 9: Load objects   ####

processed_docs = pickle.load(open("NYT_clean.sav","rb"))  #<- the processed NYT snippets
corpus_tfidf = pickle.load(open("corpus_tfidf.sav","rb"))
bow_corpus = pickle.load(open("bow_corpus.sav","rb"))
dictionary = pickle.load(open("dictionary.sav","rb"))
DTM = pickle.load(open("DTM.sav","rb"))

# Load NYT article data from original file.
NYT = pd.read_csv('NYT_article_data.csv')

# Load pre-saved word counts array we pickled.
word_counts_array = pickle.load(open("word_counts_array.sav","rb"))




In [ ]:
#=================================================-
#### Slide 29: Running LdaMulticore  ####

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics = 5, 
                                             id2word = dictionary, 
                                             workers = 4, 
                                             passes = 2)
print(lda_model_tfidf)




In [ ]:
#=================================================-
#### Slide 30: LDA output  ####

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))




In [ ]:
#=================================================-
#### Slide 31: LDA output - cont'd  ####

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))




In [ ]:
#=================================================-
#### Slide 32: Classify our documents within topics  ####

# Now, we can check where each of our documents would be classified.
# Let's look at our first document as an example:
    
print(processed_docs[0])




In [ ]:
#=================================================-
#### Slide 33: Classify our documents within topics - cont'd  ####

for index, score in sorted(lda_model_tfidf[corpus_tfidf[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))




In [ ]:
#=================================================-
#### Slide 34: Exercise 1  ####






In [ ]:
#=================================================-
#### Slide 39: Calculate topic coherence  ####

# Compute Coherence Score using c_v.
coherence_model_lda = CoherenceModel(model = lda_model_tfidf, texts = processed_docs, dictionary = dictionary, coherence = 'c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)




In [ ]:
#=================================================-
#### Slide 41: Convenience function  ####

def compute_coherence_values(dictionary, corpus, texts, limit, start = 2, step = 3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.LdaMulticore(corpus = corpus, 
                                           id2word = dictionary, num_topics = num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model = model, texts = texts,
                                        dictionary = dictionary, coherence = 'c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values




In [ ]:
#=================================================-
#### Slide 42: Run compute_coherence_values function  ####

np.random.seed(1)
model_list, coherence_values = compute_coherence_values(dictionary = dictionary, corpus = corpus_tfidf, texts = processed_docs, start = 2, limit = 40, step = 6)
# Plot graph of topic list.
# Show graph.
import matplotlib.pyplot as plt
limit = 40; start = 2; step = 6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc = 'best')
plt.show()




In [ ]:
#=================================================-
#### Slide 43: Save results as a pickle  ####

pickle.dump(lda_model_tfidf, open('lda_model_tfidf.sav', 'wb'))




In [ ]:
#=================================================-
#### Slide 45: Exercise 2  ####






In [ ]:
#=================================================-
#### Slide 49: Visualize topics generated with LDA  ####

# Prepare LDA vis object by providing:
vis = pyLDAvis.gensim.prepare(lda_model_tfidf,   #<- model object
                              corpus_tfidf,      #<- corpus object
                              dictionary)        #<- dictionary object
                              
# The function takes `vis` object that we prepared above as the main argument.
pyLDAvis.display(vis)




In [ ]:
#=================================================-
#### Slide 59: Exercise 3  ####






In [ ]:
#=================================================-
#### Slide 62: Get topic probabilities for a document  ####

# Select the index of the document in corpus.
doc_num = 0
# Extract the vector of tf_idf weights for the document.
doc_vec = corpus_tfidf[doc_num]  
print(doc_vec)

# Extract topic probabilities for that document.
doc_topics = lda_model_tfidf.get_document_topics(doc_vec)
print(doc_topics)




In [ ]:
#=================================================-
#### Slide 63: Get topic probabilities for a document (cont'd)  ####

# Initialize maximum probability score.
max_prob = 0    
# Initialize best topic.
best_topic = 0

# Loop over all topics for the document.
for topic in doc_topics:
    
    if max_prob <= topic[1]:  #<- if current topic's probability is as high as max
        max_prob = topic[1]   #<- make current topic's probability the new max
        best_topic = topic[0] #<- make current topic best

# Create a tuple with information we just got.
doc_topic_pair = (doc_num, best_topic, max_prob)
print(doc_topic_pair)




In [ ]:
#=================================================-
#### Slide 64: Get topic probabilities for a document (cont'd)  ####

# Put it all together into a function that returns a tuple
# with the index of the document, the best fit topic, and its probability.
def GetDocTopicPair(doc_num, corpus, lda_model_tfidf):
    # Extract the vector of tf_idf weights for the document.
    doc_vec = corpus[doc_num]  
    # Extract topic probabilities for that document.
    doc_topics = lda_model_tfidf.get_document_topics(doc_vec)
    max_prob = 0
    best_topic = 0
    for topic in doc_topics:
      if max_prob <= topic[1]:
            max_prob = topic[1]
            best_topic = topic[0]
      doc_topic_pair = (doc_num, best_topic, max_prob)
    return(doc_topic_pair)




In [ ]:
#=================================================-
#### Slide 65: Get topic probabilities for all documents  ####

# Create an empty list of the same length as the number of documents.
doc_topic_pairs = [None]*dictionary.num_docs

# Loop through a range of document indices.
for i in range(dictionary.num_docs):
    # For each document index, get the document-topic tuple.
    doc_topic_pairs[i] = GetDocTopicPair(i, corpus_tfidf, lda_model_tfidf)
    
print(doc_topic_pairs[:4])




In [ ]:
#=================================================-
#### Slide 66: Create a data frame with doc-topic assignments  ####

# Make a dataframe out of a list of tuples.
doc_topic_pairs_df = pd.DataFrame(doc_topic_pairs)

# Assign column names to the dataframe.
doc_topic_pairs_df.columns = ["doc_id", "best_topic", "best_probability"]
print(doc_topic_pairs_df.head())




In [ ]:
#=================================================-
#### Slide 67: Matching document ids to NYT data  ####

# Find indices of articles that we kept.
valid_snippets = np.where(word_counts_array >= 5)[0]
print(valid_snippets[120:125])





In [ ]:
#=================================================-
#### Slide 68: Matching document ids to NYT data - cont'd  ####

# Now assign the index of the original article to be the index of the dataframe.
doc_topic_pairs_df.index = valid_snippets
print(doc_topic_pairs_df.iloc[120:125, ])




In [ ]:
#=================================================-
#### Slide 69: Inspect documents for a given topic  ####

# Filter and sort all documents assigned to topic 3 by probability in descending order.
topic3_docs = doc_topic_pairs_df.query("best_topic==2")
topic3_docs = topic3_docs.sort_values(by = "best_probability", ascending = False)
print(topic3_docs.head())

# Let's see how many documents were assigned to that topic.
print(topic3_docs.shape)





In [ ]:
#=================================================-
#### Slide 70: Inspect documents for a given topic (cont'd)  ####

# Let's get the top 10 documents that were assigned to that topic.
top_10 = topic3_docs.index[0:10,]

# Inspect the top 10 documents in topic 3.
NYT_articles_topic3 = NYT.loc[top_10, :]
print(NYT_articles_topic3[['headline']])





In [ ]:
#=================================================-
#### Slide 71: Save LDA visualization to HTML file  ####

# Save the plot as a self-contained HTML file.
pyLDAvis.save_html(vis, plot_dir+"/NYT_LDAvis.html")




In [ ]:
#=================================================-
#### Slide 72: Save results as a pickle  ####

pickle.dump(doc_topic_pairs_df, open('doc_topic_pairs_df.sav','wb'))


